## Processing individuals for Yiqi

Yiqi has request that we derive:

1.  All retweets made by one set of users after 5-01-21 (altersafter05012021)
2.  All replies to another set of users after 5-01-21 (egosafter05012021)


In [ ]:


# Wrapping the above on a function
import math
import time

def query_user(account,query, user, start_date = None):
        done = False
        user_tweet_count = 0
        error_count = 0
        token = None
        #This is just to provide some feedback for (roughly) every 10% of a users tweets recovered
        counter_increment = 50
        
        while not done:
            error = False
            try:
                result = client.search_all_tweets(query=query,max_results=500,next_token = token, 
                    start_time = start_date,
                    place_fields="full_name,country_code",
                    expansions = "referenced_tweets.id,in_reply_to_user_id,entities.mentions.username,referenced_tweets.id.author_id",
                    user_fields = "description",
                    tweet_fields = "text,created_at,conversation_id,entities,in_reply_to_user_id,public_metrics,referenced_tweets")
            
            except Exception as e:
                print(f"Error: {e}")
                error = True
            if error or (len(result.errors) > 0 and result.meta['result_count'] == 0):
                print(f"{result.errors[0]} and {len(result)-1} others")
                
                #presume rate limit, retry three times with incremental backoff
                if error_count ==3:
                    print("Too many errors, bailing")
                    done = True
                    break
                error_count+=1
                print(f"Sleep for {(backoff ** error_count)/60} minutes")
                time.sleep(backoff ** error_count)
            else:
                error_count = 0
                token = result.meta.get("next_token")
                if not token:
                    done = True
                data = merge_data(result.data,result.includes)
                if not data:
                    print(f"No data for {user} : {result.meta})")
                
                write_user_data(account,user,data, done)
                old_count = user_tweet_count
                user_tweet_count+=result.meta["result_count"]
                if math.floor(user_tweet_count/counter_increment) > math.floor(old_count/counter_increment):
                    print(".",end="")
            time.sleep(3)
        print()
        return result




In [ ]:
import datetime
alters = pd.read_csv("../../../data/alters_for_yiqi.csv",dtype={0:str})
alter_list = alters.all_alters.to_list()
dt = datetime.datetime(2021,5,20)
for a in alter_list:
    q = f"from:{a} (is:retweet OR is:reply)"
    x = query_user("CNN",q,a,dt)



8-04-22

We stopped with: 330826792_CNN
